<a href="https://colab.research.google.com/github/natanelDaniel/code-knowledge-editing/blob/main/codeEditing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Clone only once.

In [ ]:
%cd /content/drive/MyDrive
!git clone https://github.com/zjunlp/EasyEdit.git

/content/drive/MyDrive
fatal: destination path 'EasyEdit' already exists and is not an empty directory.


In [1]:
%cd /content/drive/MyDrive/EasyEdit

/content/drive/MyDrive/EasyEdit


modifications:
1.in the requierments.txt - remove the exact version of PyYAML

In [3]:
!pip install -r requirements.txt

In [4]:
from huggingface_hub import login

# This will prompt you to enter your token securely.
login()

In [ ]:
data = {
    "name": 'Lahti Town Hall',
    "question": 'Who was the architect behind the design of Lahti Town Hall?',
    "old_approach": "Eliel Saarinen",
    "new_approach": "Alfred Lahti",
    "subject": "Lahti Town Hall"
}
prompt = data["question"]
ground_truth = data["old_approach"]
target_new = data["new_approach"]

In [ ]:
# Data from extracted_deprecations.json
data = {
    "name": "Removal of np.nbytes",
    "question": "How can I find the size in bytes of a single element of a NumPy array?",
    "old_approach": "np.nbytes()",
    "new_approach": "np.dtype(<dtype>).itemsize",
    "subject": "size in bytes of a single element of a NumPy array?"
}
prompt = data["question"]
ground_truth = data["old_approach"]
target_new = data["new_approach"]

In [2]:
data = {
    "name": "US Presidential Transition",
    "question": "Who is the current President of the United States?",
    "old_approach": "Joe Biden",
    "new_approach": "Donald Trump",
    "subject": "current President of the United States",
    "completion_prompt": "The current President of the United States is"
}


prompt = data["question"]
ground_truth = data["old_approach"]
target_new = data["new_approach"]
completion_prompt = data["completion_prompt"]

In [2]:
data = {
    "name": "Pandas Append Deprecation",
    "question": "How do I add a row or another DataFrame to an existing DataFrame?",
    "old_approach": "df.append(other_df)",
    "new_approach": "pd.concat([df, other_df])",
    "subject": "add a row or another DataFrame to an existing DataFrame",
    "completion_prompt": "To combine two DataFrames, the recommended way is"
}

prompt = data["question"]
ground_truth = data["old_approach"]
target_new = data["new_approach"]
completion_prompt = data["completion_prompt"]

Pre-Editing check

In [3]:
from transformers import LlamaTokenizer
from transformers import LlamaForCausalLM
tokenizer = LlamaTokenizer.from_pretrained('meta-llama/llama-2-7b-chat-hf',)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

device = 0
model = LlamaForCausalLM.from_pretrained('meta-llama/llama-2-7b-chat-hf').to(f'cuda:{device}')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def test_model(model, tokenizer, test_prompt, device):
    inputs = tokenizer(test_prompt, return_tensors="pt", padding=True).to(device)
    # Use greedy search to see the most likely pre-trained response
    output = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask, # הוספת המסיכה
        max_new_tokens=80,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    print("Response:", tokenizer.decode(output[0], skip_special_tokens=True))


In [4]:
import json
from easyeditor import BaseEditor, ROMEHyperParams, MENDHyperParams, MEMITHyperParams
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
torch.cuda.empty_cache()
# Define the model and method
model_name = 'llama-7b'
method_name = 'ROME'

try:
    hparams = ROMEHyperParams.from_hparams(f'./hparams/{method_name}/{model_name}')
except FileNotFoundError:
    print(f"Error: Hyperparameters file not found for {model_name} with {method_name}.")
    exit()


/usr/local/lib/python3.12/dist-packages/timm/models/hub.py:4: FutureWarning: Importing from timm.models.hub is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [6]:
print("--- STARTING PRE-EDIT COMPLETION TEST ---")

test_model(model, tokenizer, completion_prompt, hparams.device)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


--- STARTING PRE-EDIT COMPLETION TEST ---

Testing Model: To combine two DataFrames, the recommended way is
Response: To combine two DataFrames, the recommended way is to use the `merge()` function. Here is an example of how to combine two DataFrames using `merge()`:
```
import pandas as pd

# create two sample DataFrames
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.


In [5]:

# Construct Language Model Editor
editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=[prompt],
    ground_truth=[ground_truth],
    target_new=[target_new],
    sequential_edit=True,
    subject=data['subject']
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Executing ROME algorithm for the update: [How do I add a row or another DataFrame to an existing DataFrame?] -> [ pd.concat([df, other_df])]
Cached context templates ['{}', 'The 20. {}', 'The following are examples. {}', 'Therefore, in this. {}', 'Therefore, we will. {}', "Because I'm. {}", 'Because it is important. {}', 'I am a . {}', "I'm just. {}", "You've got. {}", 'You are here:. {}', 'The 10 Best Places to Visit. {}', 'The following are some of the most frequently asked. {}', 'Therefore, I will continue to fight for the. {}', 'Therefore, in order to make your business successful. {}', 'Because of their small size, they have a. {}', 'Because of the high demand for the vacc. {}', 'I was at a meeting with a group of. {}', "I'm a 50 year old. {}", 'You are here: Home > Products >. {}', 'You are here: Home / Blog /. {}']
Computing left vector (u)...
Selected u projection object add a row or another DataFrame to an existing DataFrame
Left vector shape: torch.Size([11008])
Computing rig

100%|██████████| 1/1 [00:23<00:00, 23.37s/it]

loss 0.047 = 0.024 + 0.022 + 0.001 avg prob of [ pd.concat([df, other_df])] 0.9763084053993225
Delta norm: 16.134944915771484
Change in target norm: 4.033736228942871 to 16.622034072875977 => 12.588297843933105
Division Factor: 3.4085917472839355
Right vector norm: 4.733610153198242
Right vector shape: torch.Size([4096])
Deltas successfully computed for ['model.layers.5.mlp.down_proj.weight']
New weights successfully inserted into ['model.layers.5.mlp.down_proj.weight']
Metrics Summary:  {'pre': {'rewrite_acc': np.float64(0.4)}, 'post': {'rewrite_acc': np.float64(1.0)}}


In [6]:
print("--- STARTING POST-EDIT COMPLETION TEST ---")
test_model(edited_model, editor.tok, completion_prompt, hparams.device)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


--- STARTING POST-EDIT COMPLETION TEST ---

Testing Model: To combine two DataFrames, the recommended way is
Response: To combine two DataFrames, the recommended way is to use the `concat` method. Here is an example:
```
import pandas as pd

df1 = pd.DataFrame({'A': [1, 2, 3]})
df2 = pd.DataFrame({'B': [4, 5, 6]})

result = pd.concat([df1, df2], axis=0


In [19]:
import torch
import gc
# edited_model.to('cpu')
# del edited_model
# model.to('cpu')
# del model

# Force garbage collection
gc.collect()

# Now clear the cache
torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()

In [9]:
import torch
import gc

# 1. פונקציה שסורקת את כל האובייקטים בזיכרון ומוחקת Tensors
def clear_gpu_memory():
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                obj.cpu() # מעביר ל-CPU לפני המחיקה
                del obj
        except:
            pass

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect() # פקודה נוספת לניקוי זיכרון משותף

clear_gpu_memory()

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1136: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(obj, torch.Tensor)
/tmp/ipython-input-1145515470.py:8: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):


In [17]:
import torch
import gc

def list_gpu_tensors():
    found = False
    for obj in gc.get_objects():
        try:
            # בודק אם האובייקט הוא Tensor שנמצא על ה-GPU
            if torch.is_tensor(obj) and obj.is_cuda:
                size_mb = obj.element_size() * obj.nelement() / 1e6
                found = True
        except:
            pass
    if not found:
        print("לא נמצאו Tensors גלויים בזיכרון. ייתכן והזיכרון תפוס על ידי Context של CUDA או אובייקטים פנימיים.")

list_gpu_tensors()
# פונקציה למחיקה אקטיבית של כל מה שנמצא בסריקה
def kill_all_gpu_tensors():
    count = 0
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) and obj.is_cuda:
                # שינוי ה-data לטנסור ריק וקטן על ה-CPU
                obj.data = torch.tensor([0])
                count += 1
        except:
            pass

    gc.collect()
    torch.cuda.empty_cache()
    print(f"Done! Cleaned {count} tensors from GPU.")

kill_all_gpu_tensors()

Done! Cleaned 292 tensors from GPU.


In [18]:
!nvidia-smi

Thu Dec 18 10:47:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             51W /  400W |     573MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----